In [3]:
import tensorflow as tf
import numpy as np
import time 
from math import pi as Pi

batch_size = 2
MaxNAtom = 7
MaxNeigh = 7 
xyzs = np.random.random((batch_size,MaxNAtom,3))*7.0 - 5.0
Zs = np.random.randint(8,size=(batch_size,MaxNAtom,1)) # some z are zero. 
print Zs
print xyzs

[[[4]
  [2]
  [4]
  [5]
  [2]
  [6]
  [5]]

 [[7]
  [7]
  [7]
  [6]
  [6]
  [0]
  [4]]]
[[[ 1.91431642 -2.58644125 -4.52399429]
  [ 1.66216634  1.91478373 -3.1754777 ]
  [-1.37756934 -3.97945781 -3.61607495]
  [-2.76494531 -0.74811043  0.0862    ]
  [-2.96312376 -4.93497626 -3.54723092]
  [-0.15330327  1.72761479 -2.95415722]
  [-3.2569135  -4.5694895  -2.04913778]]

 [[-0.93503135  1.48367244 -1.4673057 ]
  [-1.92046654 -3.04796543 -2.20874811]
  [ 0.46850553  0.17085381  0.34586542]
  [-4.21028859 -4.26402036 -3.66641393]
  [-4.34885968 -2.41807983  1.67236981]
  [-2.83175343  1.93669575 -0.80385302]
  [-1.80028721  1.90769661  0.02631152]]]


In [ ]:
np.linalg.norm(xyzs[0,2] - xyzs[0,6])

In [4]:
def RotToQuat(axes_):
    """
    axes is a ... X 3 3 tensor of axes
    this generates a ... X 4 tensor of quaternions.
    which are 1:1 with those axes.
    """
    w = (1./2.)*tf.sqrt(tf.abs(1 + axes_[...,0, 0] + axes_[...,1, 1] + axes_[...,2, 2]))
    x = tf.sign(axes_[...,2, 1] - axes_[...,1, 2])*tf.abs(0.5*tf.sqrt(tf.abs(1.0 + axes_[...,0, 0] - axes_[...,1, 1] - axes_[...,2, 2])))
    y = tf.sign(axes_[...,0, 2] - axes_[...,2, 0])*tf.abs(0.5*tf.sqrt(tf.abs(1.0 - axes_[...,0, 0] + axes_[...,1, 1] - axes_[...,2, 2])))
    z = tf.sign(axes_[...,1, 0] - axes_[...,0, 1])*tf.abs(0.5*tf.sqrt(tf.abs(1.0 - axes_[...,0, 0] - axes_[...,1, 1] + axes_[...,2, 2])))
    return tf.stack([w,x,y,z],axis=-1)

def QuatToRot(q):
    """
    a_ ... X 4 tensor of quaternions
    this generates a ... X 3 X 3 of rotation matrices.
    """
    tmp=tf.stack([1 - 2.*(q[...,2]*q[...,2] + q[...,3]*q[...,3]), 2*(q[...,1]*q[...,2] - q[...,3]*q[...,0]),
    2*(q[...,1]*q[...,3] + q[...,2]*q[...,0]),2*(q[...,1]*q[...,2] + q[...,3]*q[...,0]), 1 - 2.*(q[...,1]*q[...,1] + q[...,3]*q[...,3]),
    2*(q[...,2]*q[...,3] - q[...,1]*q[...,0]),2*(q[...,1]*q[...,3] - q[...,2]*q[...,0]), 2*(q[...,2]*q[...,3] + q[...,1]*q[...,0]),
    1 - 2.*(q[...,1]*q[...,1] + q[...,2]*q[...,2])],axis=-1)
    return tf.reshape(tmp,[-1,3,3])

def safe_normalize(x_):
    nrm = tf.clip_by_value(tf.norm(x_,axis=-1,keepdims=True),1e-36,1e36)
    nrm_ok = tf.logical_and(tf.not_equal(nrm,0.),tf.logical_not(tf.is_nan(nrm)))
    safe_nrm = tf.where(nrm_ok,nrm,tf.ones_like(nrm))
    return x_*tf.where(nrm_ok,1.0/safe_nrm,tf.zeros_like(nrm))

def safe_inv_norm(x_):
    nrm = tf.clip_by_value(tf.norm(x_,axis=-1,keepdims=True),1e-36,1e36)
    nrm_ok = tf.logical_and(tf.not_equal(nrm,0.),tf.logical_not(tf.is_nan(nrm)))
    safe_nrm = tf.where(nrm_ok,nrm,tf.ones_like(nrm))
    return tf.where(nrm_ok,1.0/safe_nrm,tf.zeros_like(nrm))

def TF_AxisAngleRotation(axes, vecs, psis):
    """Returns a batch of uniform rotation matrices, and the angles of each.
    Finds random unit vector and then random angle around it.

    Args:     sz_: number of rotation matrices     max_dist: maximum
    rotation in units of 2*Pi
    """
    ct = tf.cos(psis)
    st = tf.sin(psis)
    omct = 1.0 - ct
    sz_ = tf.shape(axes)
    matrices = tf.reshape(
        tf.stack(
            [
                ct + axes[:, 0] * axes[:, 0] * omct,
                axes[:, 0] * axes[:, 1] * omct - axes[:, 2] * st,
                axes[:, 0] * axes[:, 2] * omct + axes[:, 1] * st,
                axes[:, 1] * axes[:, 0] * omct + axes[:, 2] * st,
                ct + axes[:, 1] * axes[:, 1] * omct,
                axes[:, 1] * axes[:, 2] * omct - axes[:, 0] * st,
                axes[:, 2] * axes[:, 0] * omct - axes[:, 1] * st,
                axes[:, 2] * axes[:, 1] * omct + axes[:, 0] * st,
                ct + axes[:, 2] * axes[:, 2] * omct
            ],
            axis=-1), [sz_[0], 3, 3])
    return tf.einsum('ijk,ik->ij', matrices, vecs)

def PointQuats(dxyzs,z1z2): 
    """This version returns two sets of axes for nearest and next-nearest
    neighbor. If the energy from both these representations is averaged the
    result will be permutationally invariant (WRT nearest-next-nearest
    motion) and rotationally invariant.

    Args:
        dxyz: a nMol X maxNatom X maxNeigh X 3 tensor of
            atoms. (differenced from center of embedding
        sparse_mask:  nMol X maxNatom X maxNeigh X 1 sparse mask
    """
    # Append orthogonal axes to dxyzs
    argshape = tf.shape(dxyzs)
    realdata = tf.reshape(dxyzs,(argshape[0] * argshape[1], argshape[2], 3))
    z1z2r = tf.reshape(z1z2,(argshape[0] * argshape[1], argshape[2], 2))
    rcut = 5.0
    axis_cutoff = rcut*rcut

    weightstore = []
    cuttore = []
    axtore = []

    # Only do upper diagonal. also sort z. 
    perms = []
    for i in range(MaxNeigh): 
        perms = perms + [[i,j] for j in range(i+1,MaxNeigh)]
    for perm in perms:
        v1 = tf.reshape(dxyzs[:, :, perm[0], :],(argshape[0] * argshape[1], 3)) + tf.constant(
                            np.array([1e-23, 0., 0.]), dtype=tf.float64)
        v2 = tf.reshape(dxyzs[:, :, perm[1], :],(argshape[0] * argshape[1], 3)) + tf.constant(
                            np.array([0., 1e-23, 0.]), dtype=tf.float64)
        w1 = tf.reshape(tf.reduce_sum(dxyzs[:, :, perm[0], :] * dxyzs[:, :, perm[0], :],
                axis=-1), (argshape[0] * argshape[1], 1))
        w2 = tf.reshape(tf.reduce_sum(dxyzs[:, :, perm[1], :] * dxyzs[:, :, perm[1], :],
                axis=-1), (argshape[0] * argshape[1], 1))
        
        mustmsk1 = tf.logical_or(tf.equal(z1z2r[:,perm[0],:1],0),tf.equal(z1z2r[:,perm[0],1:],0))
        mustmsk2 = tf.logical_or(mustmsk1,tf.equal(z1z2r[:,perm[1],1:],0))
                
        v1n = safe_inv_norm(v1) * v1
        v2n = safe_inv_norm(v2) * v2
        w3p = tf.abs(tf.reduce_sum(v1n * v2n, axis=-1)[..., tf.newaxis])

        v3 = tf.cross(v1n, v2n)
        posz = tf.tile(tf.greater(tf.reduce_sum(v3*tf.constant([[1.,1.,1.]],dtype=tf.float64),keepdims=True,axis=-1),0.),[1,3])
        v3 = tf.where(posz,v3,-1.*v3)
        v3 *= safe_inv_norm(v3)

        cw1 = tf.where(tf.less(w1, axis_cutoff), tf.cos(w1 / axis_cutoff * Pi / 2.0),
            tf.zeros_like(w1))
        cw2 = tf.where(tf.less(w2, axis_cutoff), tf.cos(w2 / axis_cutoff * Pi / 2.0),
            tf.zeros_like(w2))
        ca = tf.where(tf.greater(w3p, 0.92), tf.zeros_like(w3p),tf.cos(w3p / 0.92 * Pi / 2.))
        Cuts = tf.where(mustmsk2,tf.zeros_like(cw1), (cw1 * cw2 * ca + 1e-19))
        
        # Compute the average of v1, v2, and their projections onto the
        # plane.
        v_av = (v1n + v2n) / 2.0
        v_av *= safe_inv_norm(v_av)

        # Rotate pi/4 cw and ccw to obtain v1,v2
        first = TF_AxisAngleRotation(v3, v_av, tf.constant(Pi / 4., dtype=tf.float64))
        second = TF_AxisAngleRotation(v3, v_av,tf.constant(-Pi / 4., dtype=tf.float64))
        vs = tf.concat([first[:, tf.newaxis, :], second[:, tf.newaxis, :],v3[:, tf.newaxis, :]],axis=1)
        axtore.append(vs)
        weightstore.append(Cuts)

    axs = tf.stack(axtore, axis=-3)
    ws = tf.stack(weightstore, axis=-2) 
    dn = tf.clip_by_value(tf.reduce_sum(ws, axis=1, keepdims=True), 1e-19, 1e19)
    wws = tf.where(tf.greater_equal(ws, 0.), ws / (dn), tf.zeros_like(ws))
    
    qs = RotToQuat(axs)
    return qs,wws

In [5]:
def PowerMethod(mat_,v_): 
    """
    finds the dominant eigenvector of a matrix by the power method. 
    also returns resdiuum.   
    Args: 
        mat_: a matrix 
        v_: a guess eigenvector. 
    """
    i0 = tf.constant(0)
    v = safe_normalize(v_ + 1e-19)
    cond = lambda i, m: i < 35
    body = lambda i, vx: [i+1,tf.stop_gradient(safe_normalize(tf.einsum('ijk,ik->ij',mat_,vx)))]
    i_f, vf = tf.while_loop(
        cond, body, loop_vars=[i0, v],
        shape_invariants=[i0.get_shape(), v_.get_shape()])
    diff = vf - safe_normalize(tf.einsum('ijk,ik->ij',mat_,vf))   
    return vf, tf.reduce_sum(diff*diff,axis=-1)

def AverageQuatMat(quats,weights): 
    """
    quats is a [...,nquat,4] tensor of quaternions. 
    weights is a [...,nquat,1] tensor of weights. 
    constructs properly averaged quat as per 
    https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/20070017872.pdf
    """
    nquat = tf.shape(quats)[1]
    qr = tf.reshape(quats,(-1,nquat,4))
    wr = tf.reshape(weights,(-1,nquat,1))
    wq = qr*wr
    todiag = tf.matmul(tf.transpose(qr,perm=(0,2,1)),wq)
    return todiag

def TF_RotationBatch(thetas, phis, psis):
    """Returns a batch of uniform rotation matrices, and the angles of each.
    Finds random unit vector and then random angle around it.

    Args:     sz_: number of rotation matrices     max_dist: maximum
    rotation in units of 2*Pi
    """
    Pi = 3.14159265359
    sz_ = tf.shape(thetas)
    axes = tf.zeros(shape=sz_ + [3])
    axes = tf.stack(
        [
            tf.sin(thetas) * tf.cos(phis),
            tf.sin(thetas) * tf.sin(phis),
            tf.cos(thetas)
        ],
        axis=-1)
    ct = tf.cos(psis)
    st = tf.sin(psis)
    omct = 1.0 - ct
    matrices = tf.reshape(
        tf.stack(
            [
                ct + axes[:, 0] * axes[:, 0] * omct,
                axes[:, 0] * axes[:, 1] * omct - axes[:, 2] * st,
                axes[:, 0] * axes[:, 2] * omct + axes[:, 1] * st,
                axes[:, 1] * axes[:, 0] * omct + axes[:, 2] * st,
                ct + axes[:, 1] * axes[:, 1] * omct,
                axes[:, 1] * axes[:, 2] * omct - axes[:, 0] * st,
                axes[:, 2] * axes[:, 0] * omct - axes[:, 1] * st,
                axes[:, 2] * axes[:, 1] * omct + axes[:, 0] * st,
                ct + axes[:, 2] * axes[:, 2] * omct
            ],
            axis=-1), [sz_[0], 3, 3])
    return matrices


In [ ]:
with tf.Graph().as_default():        
    xyzs_tf = tf.Variable(xyzs)
    ZTF = tf.Variable(Zs)
    
    #thetas = tf.acos(2.0 * tf.random_uniform([batch_size], dtype=tf.float64) - 1.0)
    #phis = tf.random_uniform([batch_size], dtype=tf.float64) * 2 * Pi
    #psis = tf.random_uniform([batch_size], dtype=tf.float64) * 2 * Pi
    thetas = tf.acos(2.0 * tf.random_uniform([batch_size], dtype=tf.float64) - 1.0)
    phis = tf.random_uniform([batch_size], dtype=tf.float64) * 2 * Pi
    psis = tf.random_uniform([batch_size], dtype=tf.float64) * 2 * Pi    
    matrices = TF_RotationBatch(thetas, phis, psis)
    xyzs_shifted = xyzs_tf - xyzs_tf[:, 0, :][:,tf.newaxis, :]
    tmpxyzs = tf.einsum('ijk,ikl->ijl', xyzs_shifted, matrices)
    
    testtf = tf.constant([[[1., 2, 3], [2, 5, 6], [3, 6, 3]]],dtype=tf.float64)
    testtf2 = tf.constant([[1,1,1]],dtype=tf.float64)
    
    dxyzs = tf.expand_dims(tmpxyzs, axis=2) - tf.expand_dims(tmpxyzs, axis=1) # mol X MaxNAtom X MaxNeigh X3
    # Compute quats. 
    z1 = tf.tile(tf.expand_dims(ZTF, axis=2),(1,1,MaxNeigh,1))
    z2 = tf.tile(tf.expand_dims(ZTF, axis=1),(1,MaxNeigh,1,1))
    z1z2 = tf.concat([z1,z2],axis=-1)
    quats, weights = PointQuats(dxyzs,z1z2)
    nquat = tf.shape(quats)[1]
    aqm = AverageQuatMat(quats,weights)
    #aqm = tf.eye(4,batch_shape=tf.shape(quats)[:1],dtype=tf.float64)
    aquats, resids = PowerMethod(aqm,quats[:,0,:])
    rots = tf.reshape(QuatToRot(aquats),(batch_size, MaxNAtom,3,3))
    # now transform the coords. 
    cdxyzs = tf.reshape(tf.einsum('mijk,milk->mijl', dxyzs, rots),(batch_size, MaxNAtom, MaxNeigh,3))

    init = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)

    #print sess.run(PowerMethod(testtf,testtf2))
    #print sess.run(xyzs_tf)
    #print sess.run(quats)
    #print sess.run(aquats)
    Ws = sess.run(weights) 
    D = sess.run(dxyzs)
    CD = sess.run(cdxyzs)
    #print sess.run(aquats).shape
    #print sess.run(cdxyzs).shape
    #rmats = sess.run(rots)

    print sess.run(tf.gradients(tf.reduce_sum(dxyzs*dxyzs,axis=2),psis))
    print sess.run(tf.gradients(tf.reduce_sum(cdxyzs*cdxyzs,axis=2),psis))
    

In [ ]:
Ws

In [ ]:
print Ws.shape
print Ws[2]
with tf.Graph().as_default(): 
    print sess.run(tf.gradients(weights,psis))

In [ ]:
with tf.Graph().as_default(): 
    Xqs,Xmat,Xaq = sess.run([quats[0,0,:],aqm[0],aquats])
    print Xqs,Xmat,Xaq[0]
    print sess.run(tf.gradients(aquats[2,2],psis))

In [ ]:
print Zs[0]
print Ws[2]

In [ ]:
print D[0,0,0],D[0,0,1]
print CD[0,0,0],CD[0,0,1]
print Ws[0,0,0],Ws[0,1,0]
print np.linalg.norm(D[0,0,0]-D[0,0,1])
print np.linalg.norm(CD[0,0,0]-CD[0,0,1])

In [ ]:
print D[0,1,3],D[0,1,2]
print CD[0,1,3],CD[0,1,2]
print np.linalg.norm(D[0,1,3]-D[0,1,2])
print np.linalg.norm(CD[0,1,3]-CD[0,1,2])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(D[0,0,0], D[0,0,1], D[0,0,0])


In [ ]:
def polykern_sr_9_11(r):
    """
    Only LR cutoff.
    """
    LRInner = 9.
    LROuter = 11.
    a = 1595.69
    b = -668.25
    c = 104.625
    d = -7.25
    e = 0.1875
    r2 = r * r
    r3 = r2 * r
    r4 = r3 * r
    kernm1 = tf.where(tf.less(r, LRInner), tf.clip_by_value(1.0/(1e-19+r),-1e19,1e19),
                (a + b * r + c * r2 + d * r3 + e * r4) /r)
    kern0 = tf.where(tf.less_equal(r,0.),tf.zeros_like(r),kernm1)
    kern = tf.where(tf.greater(r, LROuter), tf.zeros_like(r),kern0)
    return kern

In [ ]:
with tf.Graph().as_default():        
    rs = tf.Variable(np.linspace(0.0,30.0,300),dtype=tf.float64)
    init = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    print sess.run(polykern_sr_9_11(rs))

In [ ]:
np.linspace(0.0,40.0,100)

In [6]:
with tf.Graph().as_default():        
    rs = tf.Variable(np.linspace(0.0,30.0,300),dtype=tf.float64)
    init = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    
def QuatToRot(q):
    """
    a_ ... X 4 tensor of quaternions
    this generates a ... X 3 X 3 of rotation matrices.
    """
    tmp=tf.stack([1 - 2.*(q[...,2]*q[...,2] + q[...,3]*q[...,3]), 2*(q[...,1]*q[...,2] - q[...,3]*q[...,0]),
    2*(q[...,1]*q[...,3] + q[...,2]*q[...,0]),2*(q[...,1]*q[...,2] + q[...,3]*q[...,0]), 1 - 2.*(q[...,1]*q[...,1] + q[...,3]*q[...,3]),
    2*(q[...,2]*q[...,3] - q[...,1]*q[...,0]),2*(q[...,1]*q[...,3] - q[...,2]*q[...,0]), 2*(q[...,2]*q[...,3] + q[...,1]*q[...,0]),
    1 - 2.*(q[...,1]*q[...,1] + q[...,2]*q[...,2])],axis=-1)
    return tf.reshape(tmp,[-1,3,3])

def VectorsToOrient(v1,v2):
    v1n = safe_normalize(v1)
    v2n = safe_normalize(v2)
    v3 = safe_normalize(tf.cross(v1n, v2n)+tf.constant(np.array([0., 0., 1e-19]), dtype=tf.float64))
    # Compute the average of v1, v2, and their projections onto the
    # plane.
    v_av = (v1n + v2n) / 2.0
    v_av = safe_normalize(v_av)
    # Rotate pi/4 cw and ccw to obtain v1,v2
    first = TF_AxisAngleRotation(v3, v_av, tf.constant(-Pi / 4., dtype=tf.float64))
    second = TF_AxisAngleRotation(v3, v_av,tf.constant(Pi / 4., dtype=tf.float64))
    vs = tf.concat([first[:, tf.newaxis, :], second[:, tf.newaxis, :],v3[:, tf.newaxis, :]],axis=1)
    return vs

In [ ]:
with tf.Graph().as_default():        
    rs = tf.Variable(np.array([[.3,.3,.3,.3]]),dtype=tf.float64)
    init = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    print sess.run(QuatToRot(rs))

In [7]:
def safe_normalize(x_):
    nrm = tf.clip_by_value(tf.norm(x_,axis=-1,keepdims=True),1e-36,1e36)
    nrm_ok = tf.logical_and(tf.not_equal(nrm,0.),tf.logical_not(tf.is_nan(nrm)))
    safe_nrm = tf.where(nrm_ok,nrm,tf.ones_like(nrm))
    return x_*tf.where(nrm_ok,1.0/safe_nrm,tf.zeros_like(nrm))
def safe_inv_norm(x_):
    nrm = tf.clip_by_value(tf.norm(x_,axis=-1,keepdims=True),1e-36,1e36)
    nrm_ok = tf.logical_and(tf.not_equal(nrm,0.),tf.logical_not(tf.is_nan(nrm)))
    safe_nrm = tf.where(nrm_ok,nrm,tf.ones_like(nrm))
    return tf.where(nrm_ok,1.0/safe_nrm,tf.zeros_like(nrm))
def safe_norm(x_):
    nrm = tf.clip_by_value(tf.norm(x_, axis=-1, keepdims=True), 1e-36, 1e36)
    nrm_ok = tf.logical_and(
        tf.not_equal(nrm, 0.), tf.logical_not(tf.is_nan(nrm)))
    safe_nrm = tf.where(nrm_ok, nrm, tf.zeros_like(nrm))
    return safe_nrm
def RotToQuat(axes_):
    """
    axes is a ... X 3 3 tensor of axes
    this generates a ... X 4 tensor of quaternions.
    which are 1:1 with those axes.
    """
    w = (1./2.)*tf.sqrt(1e-19+tf.abs(1 + axes_[...,0, 0] + axes_[...,1, 1] + axes_[...,2, 2]))
    x = tf.sign(axes_[...,2, 1] - axes_[...,1, 2])*tf.abs(0.5*tf.sqrt(1e-19+tf.abs(1.0 + axes_[...,0, 0] - axes_[...,1, 1] - axes_[...,2, 2])))
    y = tf.sign(axes_[...,0, 2] - axes_[...,2, 0])*tf.abs(0.5*tf.sqrt(1e-19+tf.abs(1.0 - axes_[...,0, 0] + axes_[...,1, 1] - axes_[...,2, 2])))
    z = tf.sign(axes_[...,1, 0] - axes_[...,0, 1])*tf.abs(0.5*tf.sqrt(1e-19+tf.abs(1.0 - axes_[...,0, 0] - axes_[...,1, 1] + axes_[...,2, 2])))
    return tf.stack([w,x,y,z],axis=-1)
def TF_AxisAngleRotation(axes, vecs, psis):
    """Returns a batch of uniform rotation matrices, and the angles of each.
    Finds random unit vector and then random angle around it.

    Args:     sz_: number of rotation matrices     max_dist: maximum
    rotation in units of 2*Pi
    """
    ct = tf.cos(psis)
    st = tf.sin(psis)
    omct = 1.0 - ct
    sz_ = tf.shape(axes)
    matrices = tf.reshape(
        tf.stack(
            [
                ct + axes[:, 0] * axes[:, 0] * omct,
                axes[:, 0] * axes[:, 1] * omct - axes[:, 2] * st,
                axes[:, 0] * axes[:, 2] * omct + axes[:, 1] * st,
                axes[:, 1] * axes[:, 0] * omct + axes[:, 2] * st,
                ct + axes[:, 1] * axes[:, 1] * omct,
                axes[:, 1] * axes[:, 2] * omct - axes[:, 0] * st,
                axes[:, 2] * axes[:, 0] * omct - axes[:, 1] * st,
                axes[:, 2] * axes[:, 1] * omct + axes[:, 0] * st,
                ct + axes[:, 2] * axes[:, 2] * omct
            ],
            axis=-1), [sz_[0], 3, 3])
    return tf.einsum('ijk,ik->ij', matrices, vecs)
with tf.Graph().as_default():        
    xyzs = tf.Variable(np.random.random((batch_size,MaxNAtom,3))*7.0 - 5.0)
    init = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    print sess.run(xyzs[0,:2])
    print sess.run(VectorsToOrient(xyzs[:,0],xyzs[:,1]))
    print sess.run(RotToQuat(VectorsToOrient(xyzs[:,0],xyzs[:,1])))
    print sess.run(QuatToRot(RotToQuat(VectorsToOrient(xyzs[:,0],xyzs[:,1]))))
    

[[-4.52013408 -0.22379047 -1.69580106]
 [-3.55814909  0.54129489  0.76857379]]
[[[-0.6665386  -0.19328034 -0.71997847]
  [-0.74178371  0.26789563  0.61480799]
  [ 0.07404879  0.94386156 -0.32193499]]

 [[ 0.45647495 -0.70744694  0.53958267]
  [-0.7966752  -0.05496639  0.60190308]
  [-0.39615558 -0.70462581 -0.58869621]]]
[[ 0.26430193  0.31124779 -0.75106079 -0.5188227 ]
 [ 0.45078053 -0.72459257  0.51895445 -0.04948542]]
[[[-0.6665386  -0.19328034 -0.71997847]
  [-0.74178371  0.26789563  0.61480799]
  [ 0.07404879  0.94386156 -0.32193499]]

 [[ 0.45647495 -0.70744694  0.53958267]
  [-0.7966752  -0.05496639  0.60190308]
  [-0.39615558 -0.70462581 -0.58869621]]]


In [8]:
def slerp(v0, v1, t=0.05):
    """
    Interpolate between quaternions v0 and v1.
    """
    v0 = safe_normalize(v0)
    v1 = safe_normalize(v1)
    dot = tf.reduce_sum(v0*v1,axis=-1,keepdims=True)
    # If the dot product is negative, slerp won't take
    # the shorter path. Note that v1 and -v1 are equivalent when
    # the negation is applied to all four components. Fix by
    # reversing one quaternion.
    signflip = tf.where(tf.less_equal(dot,0.),-1.*tf.ones_like(dot),tf.ones_like(dot))
    v1 *= signflip
    dot *= signflip
    # Linear answer.
    linq = safe_normalize(v0 + t*(v1-v0))
    #
    sdot = tf.clip_by_value(dot,-1.0,1.0)
    theta_0 = tf.acos(sdot)
    theta = theta_0*t
    sin_theta = tf.sin(theta)
    sin_theta_0 = tf.sin(theta_0)
    s0 = tf.cos(theta) - dot * sin_theta / (sin_theta_0+1e-19)
    s1 = sin_theta / (sin_theta_0+1e-19)
    sq = safe_normalize((s0 * v0) + (s1 * v1))
    #
    DOT_THRESHOLD = 0.9995
    tdot = tf.concat([dot,dot,dot,dot],axis=-1)
    slerpd = tf.where(tf.greater(tdot,DOT_THRESHOLD),linq,sq)
    ttiled = tf.concat([t,t,t,t],axis=-1)
    slerpdorv1 = tf.where(tf.greater(ttiled,1.0-1e-14),v1,slerpd)
    return tf.where(tf.less(ttiled,1e-14),v0,slerpdorv1)

In [9]:
with tf.Graph().as_default():        
    xyzs = tf.Variable(np.random.random((batch_size,MaxNAtom,3))*7.0 - 5.0)
    q1=RotToQuat(VectorsToOrient(xyzs[:,0],xyzs[:,1]))
    q2=RotToQuat(VectorsToOrient(xyzs[:,1],xyzs[:,2]))
    init = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    print sess.run(q1)
    print sess.run(q2)
    print sess.run(slerp(q1,q2,0.0))
    print sess.run(slerp(q1,q2,0.2))
    print sess.run(slerp(q1,q2,0.4))
    print sess.run(slerp(q1,q2,0.6))    
    print sess.run(slerp(q1,q2,1.0))        


[[ 0.10908506  0.92581498  0.00113723 -0.36189195]
 [ 0.16528367 -0.00838225 -0.77582942  0.60885118]]
[[ 0.5700113   0.51692443 -0.53746492  0.34497495]
 [ 0.0237009   0.67409075 -0.21600999 -0.70596007]]


ValueError: Can't concatenate scalars (use tf.stack instead) for 'concat_3' (op: 'ConcatV2') with input shapes: [], [], [], [], [].